# 2CSSID Workshop03. Neural networks' tools (Pytorch)

<p style='text-align: right;font-style: italic; color: red;'>Designed by: Mr. Abdelkrime Aries</p>


In [ ]:
%pip install torch

In [1]:
import torch
from torch import Tensor, nn, optim

torch.__version__

'2.5.1+cpu'

In [ ]:
%pip install pytorch-lightning
import pytorch_lightning as pl

In [2]:
import pandas     as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

pd.__version__

'2.2.3'

In [4]:
from typing import Literal, List

## I. Data preparation

In [7]:
df = pd.read_csv('data/sat.trn', delimiter=' ', header=None)

# Generate column names as 'column1', 'column2', ..., 'columnN'
df.columns = [f'column{i+1}' for i in range(df.shape[1])]

# Rename the last column to 'output'
df.columns = list(df.columns[:-1]) + ['output']

x_train = df.iloc[:, :-1].values  # Selecting all columns except the last one
y_train = df.iloc[:, -1].values  # Selecting the last column as y_train

# Normalize x_train (divide by 255)
x_train = x_train / 255.0

# Apply LabelBinarizer to y_train
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)

x_train_tensor = torch.tensor(x_train)
y_train_tensor = torch.tensor(y_train)

In [10]:
test_df = pd.read_csv('data/sat.tst', delimiter=' ', header=None)

# Generate column names as 'column1', 'column2', ..., 'columnN'
test_df.columns = [f'column{i+1}' for i in range(test_df.shape[1])]

# Rename the last column to 'output'
test_df.columns = list(test_df.columns[:-1]) + ['output']

x_test = test_df.iloc[:, :-1].values  # Selecting all columns except the last one
y_test = test_df.iloc[:, -1].values  # Selecting the last column as y_test

# Normalize x_test (divide by 255)
x_test = x_test / 255.0

# Apply LabelBinarizer to y_test
label_binarizer = LabelBinarizer()
y_test = label_binarizer.fit_transform(y_test)

x_test_tensor = torch.tensor(x_test)
y_test_tensor = torch.tensor(y_test)

## II. High level

### II.1. Sequential model

In [11]:
import torch.nn.functional as F

class MyModel(pl.LightningModule):
    def __init__(self, input_features, num_classes):
        super(MyModel, self).__init__()
        # Define the layers
        self.layer1 = nn.Linear(input_features, 10)  # (input_features, 10)
        self.layer2 = nn.Linear(10, 10)  # (10, 10)
        self.layer3 = nn.Linear(10, num_classes)  # (10, num_classes)
        
    def forward(self, x):
        # Forward pass with ReLU activations and Softmax output
        x = F.relu(self.layer1(x))  # Apply ReLU activation after the first dense layer
        x = F.relu(self.layer2(x))  # Apply ReLU activation after the second dense layer
        x = self.layer3(x)  # Output layer (no activation here)
        return F.softmax(x, dim=1)  # Apply Softmax activation along dimension 1 (for class probabilities)

# Assuming the number of features in your dataset is 5 and the number of classes is 3
input_features = x_train_tensor.shape[1]  # Number of features from your dataset (e.g., 5)
num_classes = 6  # Example for 3 output classes

# Instantiate the model
model = MyModel(input_features, num_classes)

# Print the model to see its structure
print(model)

MyModel(
  (layer1): Linear(in_features=36, out_features=10, bias=True)
  (layer2): Linear(in_features=10, out_features=10, bias=True)
  (layer3): Linear(in_features=10, out_features=6, bias=True)
)


### II.2. Model training

In [13]:
criterion = nn.CrossEntropyLoss()

# Define Adam optimizer with a learning rate of 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop

x_train_tensor = x_train_tensor.float()
num_epochs = 10  # Define the number of epochs (iterations)
for epoch in range(num_epochs):
    # Forward pass: Compute predicted y by passing x to the model
    outputs = model(x_train_tensor)
    
    # Compute the loss
    loss = criterion(outputs, y_train_tensor.argmax(dim=1))  # Convert y_train_tensor to class indices for CrossEntropyLoss
    
    # Zero the gradients before running the backward pass
    optimizer.zero_grad() ####################################!!!!!!! why we used that ??
    
    # Backward pass: Compute gradients
    loss.backward()
    
    # Optimization step: Update the model parameters
    optimizer.step()
    
    # Print the loss for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

Epoch [1/10], Loss: 1.7983129024505615
Epoch [2/10], Loss: 1.7979542016983032
Epoch [3/10], Loss: 1.7976038455963135
Epoch [4/10], Loss: 1.7972749471664429
Epoch [5/10], Loss: 1.7969673871994019
Epoch [6/10], Loss: 1.7966729402542114
Epoch [7/10], Loss: 1.79637610912323
Epoch [8/10], Loss: 1.7960841655731201
Epoch [9/10], Loss: 1.7958147525787354
Epoch [10/10], Loss: 1.795547604560852


### II.3. Model testing

In [19]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation for inference
    # Get the raw output (logits) for X_test
    outputs = model(x_test_tensor.float())  # Ensure the input is float32
    
    # Convert the output probabilities to predicted class labels
    _, predicted_classes = torch.max(outputs, dim=1)


# Convert predicted class indices back to original class labels
predicted_labels = label_binarizer.inverse_transform(predicted_classes.reshape(-1,1).cpu().numpy())

# Assuming y_test contains the true labels in their original form
y_test_labels = label_binarizer.inverse_transform(y_test_tensor.cpu().numpy())

# Print the classification report
print(classification_report(y_test_labels, predicted_labels))

              precision    recall  f1-score   support

           1       0.23      1.00      0.37       461
           2       0.00      0.00      0.00       224
           3       0.00      0.00      0.00       397
           4       0.00      0.00      0.00       211
           5       0.00      0.00      0.00       237
           7       0.00      0.00      0.00       470

    accuracy                           0.23      2000
   macro avg       0.04      0.17      0.06      2000
weighted avg       0.05      0.23      0.09      2000



c:\Users\imed\Desktop\SID\ML\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\imed\Desktop\SID\ML\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\imed\Desktop\SID\ML\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## III. High level with a custom class

### III.1. Custom Layer

In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyLayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True, activation='linear'):
        super(MyLayer, self).__init__()
        
        # Ensure that inputs and outputs are greater than 0
        assert in_features > 0, "Number of inputs must be greater than 0"
        assert out_features > 0, "Number of outputs must be greater than 0"
        
        # Define the linear layer
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        
        # Set the activation function based on the string provided
        if activation == 'relu':
            self.activation = F.relu
        elif activation == 'sigmoid':
            self.activation = torch.sigmoid
        elif activation == 'linear':
            self.activation = None  # No activation (linear)
        else:
            raise ValueError("Activation function must be one of ['relu', 'sigmoid', 'linear']")
        
        # Store out_features as a class attribute
        self._out_features = out_features
        
    def forward(self, x):
        # Apply the linear transformation
        x = self.linear(x)
        
        # Apply the activation function if any
        if self.activation:
            x = self.activation(x)
        
        return x
    
    @property
    def bias(self):
        # Access bias through the Linear layer
        return self.linear.bias
    
    @property
    def weight(self):
        # Access weight through the Linear layer
        return self.linear.weight
    
    @property
    def out_features(self):
        # Access the number of output features
        return self._out_features

In [54]:
# Must print an 'Exception' or 'AssertionError'

try:
    ml1 = MyLayer(0, 2)
except Exception as e:
    print(repr(e))

print('end')

AssertionError('Number of inputs must be greater than 0')
end


In [55]:
l2ts = [
    MyLayer(3, 2, bias=False, activation='relu'),
    MyLayer(3, 2, bias=True, activation='sigmoid'),
    MyLayer(3, 1)
    ]

XX = Tensor([[1, 2, 3], [4, 5, 6]])

for l in l2ts:
    print('===============================')
    print(l)
    print('-------------------------------')
    print('bias=', l.bias)
    weight = Tensor(l.weight)
    print('output=', l(XX))


MyLayer(
  (linear): Linear(in_features=3, out_features=2, bias=False)
)
-------------------------------
bias= None
output= tensor([[0.3790, 0.0000],
        [0.0446, 0.0000]], grad_fn=<ReluBackward0>)
MyLayer(
  (linear): Linear(in_features=3, out_features=2, bias=True)
)
-------------------------------
bias= Parameter containing:
tensor([ 0.5747, -0.0874], requires_grad=True)
output= tensor([[0.6050, 0.1856],
        [0.3526, 0.0577]], grad_fn=<SigmoidBackward0>)
MyLayer(
  (linear): Linear(in_features=3, out_features=1, bias=True)
)
-------------------------------
bias= Parameter containing:
tensor([-0.3147], requires_grad=True)
output= tensor([[-0.4266],
        [-0.1021]], grad_fn=<AddmmBackward0>)


### III.2. Custom Net

In [56]:
class MyMLP(nn.Module):
    def __init__(self):
        super(MyMLP, self).__init__()

        self.lock = False
        self.layers = nn.ModuleList()
        self.loss_fn = None
        self.optimizer = None

    def add_layer(self, layer):
        if self.lock:
            raise Exception("Cannot add layer: the model is locked.")
        
        if len(self.layers) > 0:
            previous_layer_output = self.layers[-1].out_features
            current_layer_input = layer.in_features
            if previous_layer_output != current_layer_input:
                raise Exception(f"Layer input size ({current_layer_input}) does not match previous layer output size ({previous_layer_output}).")
        
        self.layers.append(layer)
        return self

    def compile(self, num_inputs=1, num_outputs=1, bias=True, multi_class=False, learning_rate=1.0):
        if self.lock:
            raise Exception("Cannot compile: the model is locked.")
        
        if len(self.layers) > 0:
            num_inputs = self.layers[-1].out_features
        
        output_layer = MyLayer(num_inputs, num_outputs, bias=bias)
        self.add_layer(output_layer)
        
        if multi_class and num_outputs > 1:
            self.activation_fn = nn.Softmax(dim=1)
            self.loss_fn = nn.CrossEntropyLoss()
        else:
            self.activation_fn = nn.Sigmoid()
            self.loss_fn = nn.BCEWithLogitsLoss()
        
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.lock = True

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.activation_fn(x)
        return x

    def backward(self, X, Y):
        predictions = self.forward(X)
        loss = self.loss_fn(predictions, Y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.detach().numpy()

    def fit(self, X, Y, epochs):
        for epoch in range(epochs):
            loss = self.backward(X, Y)
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss}")

    def __call__(self, X, Y):
        return self.backward(X, Y)

### III.3. Model training

In [57]:
nn2 = MyMLP()
nn2.add_layer(MyLayer(x_train.shape[1], 10, activation='relu'))\
   .add_layer(MyLayer(10, 10, activation='relu'))\
   .compile(nb_out=y_train.shape[1], lr=0.01, multiclass=True)

nn2

AttributeError: 'MyLayer' object has no attribute 'in_features'

### III.4. Model testing

## IV. Low level

### IV.1. Activation functions

In [ ]:
XX = Tensor([[1., -1., 0.], [-0.5, 0.2, 5]])
sigmoid = SimpleSigmoid()
print(sigmoid(XX))
relu = SimpleReLU()
print(relu(XX))
softmax = SimpleSoftmax()
print(softmax(XX))

### IV.2. Loss functions

### IV.3. Optimization functions

### IV.4. Custom Layer

In [ ]:
# SimpleLayer in here


In [ ]:
sl = SimpleLayer(3, 2, bias=False)

sl.randomize()
sl.b, sl.W, list(sl.parameters())

### IV.5. Custom Net

In [ ]:
# Result:
# tensor([[0.8401],
#         [0.8428]], grad_fn=<MulBackward0>)
# 1.0020916
# Parameter containing:
# tensor([[0.5149],
#         [0.5659]], requires_grad=True)

nn3t = SimpleMLP()
nn3t.add_layer(SimpleLayer(2, 2, act='sigmoid'))\
    .add_layer(SimpleLayer(2, 2, act='sigmoid'))\
    .compile()

# print(nn3)

with torch.no_grad():
    nn3t.layers[0].W += torch.Tensor([[0.5, 0.3], [0.2, 0.4]])
    nn3t.layers[0].b += torch.Tensor([[-0.3, 0.5]])
    nn3t.layers[1].W += torch.Tensor([[0.3, -0.1], [0.5, -0.3]])
    nn3t.layers[1].b += torch.Tensor([[-0.3, -0.2]])
    nn3t.layers[2].W  += torch.Tensor([[0.7], [0.7]])
    nn3t.layers[2].b  += torch.Tensor([[1.]])

XX = Tensor([[2, -1], [3, 5]])
YY = Tensor([[0], [1]])

print(nn3t.forward(XX))

loss = nn3t.backward(XX, YY)

print(loss)

nn3t.layers[2].W

### IV.6. Model training

### IV.7. Model testing